### @author: ODD team
# XGBoost model- this is adapted

In [ ]:
# Import Module
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import ensemble
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from custom_datasets import CustomDataset
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import warnings
from sklearn.model_selection import KFold

/Users/seungyoo/miniforge3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [5]:
warnings.filterwarnings(action='ignore')
from sklearn.metrics import make_scorer
#define your own mse and set greater_is_better=False
mse = make_scorer(mean_squared_error,greater_is_better=False)

In [6]:
# train valid test set
df_train = pd.read_csv('../datasets/kitti_train_1.csv')
df_valid = pd.read_csv('../datasets/kitti_valid_1.csv')
df_test = pd.read_csv('../datasets/kitti_test_1.csv')

In [7]:
train = df_train.drop(['filename', 'weather','depth_min'], axis=1)
valid = df_valid.drop(['filename', 'weather','depth_min'], axis=1)
test = df_test.drop(['filename', 'weather','depth_min'], axis=1)

In [8]:
# encoding
le=LabelEncoder()
tr=le.fit_transform(train['class'].values)
vv=le.fit_transform(valid['class'].values)
ts=le.fit_transform(test['class'].values)

train['class']=tr
valid['class']=vv
test['class']=ts

In [19]:
train = train[train['zloc'] < 90].reset_index(drop=True)
valid = valid[valid['zloc'] < 90].reset_index(drop=True)
test = pd.DataFrame(test[test['zloc']< 90]).reset_index(drop=True)


X_train = train.loc[:, train.columns != 'zloc']
y_train = train.loc[:, train.columns == 'zloc']

X_valid = valid.loc[:, valid.columns != 'zloc']
y_valid = valid.loc[:, valid.columns == 'zloc']

X_test = test.loc[:, test.columns != 'zloc']
y_test = test.loc[:, test.columns == 'zloc']

In [10]:
X_train.head()

,class,xmin,ymin,xmax,ymax,angle,depth_x,depth_y,depth_mean,depth_median,depth_max,depth_mean_trim,width,height
0,3,711.87683,151.95032,809.37300,308.49020,-0.20,740,206,1.353852,1.188141,3.025311,1.258110,97.49617,156.53988
1,2,390.57785,181.78027,421.47256,200.24520,1.85,402,199,8.203781,8.042789,9.251301,8.120802,30.89471,18.46493
2,2,655.78680,190.24402,698.48170,221.83864,-1.67,688,215,5.255445,4.740215,9.392394,4.779619,42.69490,31.59462
3,2,612.12930,176.14651,729.66540,282.53200,1.55,714,223,2.592289,1.772914,9.068852,1.894969,117.53610,106.38549
4,2,283.69547,184.47151,343.14720,211.07599,1.96,289,205,5.243309,5.002338,7.594889,5.023374,59.45173,26.60448


In [11]:
# scaling
from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler().fit(X_train)
#scalery = StandardScaler().fit(y_train)

In [12]:
X_train= scalerX.transform(X_train)
#y_train_scale = scalery.transform(y_train)

X_valid= scalerX.transform(X_valid)
#y_valid_scale = scalery.transform(y_valid)

X_test= scalerX.transform(X_test)
#y_test_scale = scalery.transform(y_test)

In [118]:
#텐서화
#X_train = xgb.DMatrix(X_train_scale)
#y_train = xgb.DMatrix(y_train.values)

#X_valid = xgb.DMatrix(X_valid_scale)
#y_valid = xgb.DMatrix(y_valid.values)

#X_test = xgb.DMatrix(X_test_scale)
#y_test = xgb.DMatrix(y_test.values)

In [13]:
os.makedirs('./weights', exist_ok=True)

In [120]:

# this is parameter tuning result
"""kfol=KFold(n_splits=3,random_state=2022,shuffle=True)
xgb1 = xgb.XGBRegressor( tree_method='gpu_hist')
"""

"""
parameters = {
              'objective':['reg:squarederror'],
              'learning_rate': [0.01,1e-5], 
              'max_depth': [3,5,7,9],
              'min_child_weight': [1,3,4,5],
              'subsample': [0.6,0.7,0.8,1],
              'colsample_bytree':[i/10.0 for i in range(6,10)],
              'n_estimators': [100,200,500,750,1000]
              #'gamma':[i/10.0 for i in range(0,5)],
              #'reg_lambda':[1,0.9,0.8,0.7,0],
              #'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1, 1],
              }
"""
# another parameter tuning
"""
parameters={
    'n_estimators': [100,200,500,750,1000],
    'max_depth': [3,5,7,9],
    'min_child_weight': [1,3,5],
    'gamma':[i/10.0 for i in range(0,5)],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1, 1],
    'learning_rate': [0.01, 0.02, 0.05, 0.1]
}
"""

"""
parameters= {'colsample_bytree': [0.9], 
              'learning_rate': [0.01], 
              'max_depth': [9], 
              'min_child_weight': [3], 
              'n_estimators': [1000], 
              'objective': ['reg:squarederror'], 
              'subsample': [0.7],
              'gamma':[i/10.0 for i in range(0,5)],
              'reg_lambda':[1,0.9,0.8,0.7,0],
              'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1, 1],
              }
"""

"\nparameters= {'colsample_bytree': [0.9], \n              'learning_rate': [0.01], \n              'max_depth': [9], \n              'min_child_weight': [3], \n              'n_estimators': [1000], \n              'objective': ['reg:squarederror'], \n              'subsample': [0.7],\n              'gamma':[i/10.0 for i in range(0,5)],\n              'reg_lambda':[1,0.9,0.8,0.7,0],\n              'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1, 1],\n              }\n"

In [12]:
# using Grid Search, we can get params tuning
"""
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = kfol,
                        n_jobs = 3,
                        verbose=0
                        )                     
"""

In [ ]:
# do the grid search and print the result
"""
xgb_grid.fit(X_train,y_train,early_stopping_rounds=20,eval_metric='rmse',eval_set=[(X_valid,y_valid)],verbose=0)


print(xgb_grid.best_score_)
print(xgb_grid.best_params_)"""

In [14]:

#print(xgb_grid.best_score_)
#print(xgb_grid.best_params_)

### 5/5 
# best parameter
bestparm={'colsample_bytree': 0.9,
          'gamma': 0.3,
          'learning_rate': 0.01,
          'max_depth': 9,
          'min_child_weight': 3,
          'n_estimators': 1000,
          'objective': 'reg:squarederror',
          'reg_alpha': 1,
          'reg_lambda': 0.9,
          'subsample': 0.7}

In [84]:
"""

best_params= {'colsample_bytree': 0.9, 
              'learning_rate': 0.01, 
              'max_depth': 9, 
              'min_child_weight': 3, 
              'n_estimators': 1000, 
              'objective': 'reg:squarederror', 
              'subsample': 0.7}
"""

"\nresently parameters\n\nbest_params= {'colsample_bytree': 0.9, \n              'learning_rate': 0.01, \n              'max_depth': 9, \n              'min_child_weight': 3, \n              'n_estimators': 1000, \n              'objective': 'reg:squarederror', \n              'subsample': 0.7}\n"

In [15]:
# change the data to fit the model format
d_train=xgb.DMatrix(data=X_train,label=y_train)
d_valid=xgb.DMatrix(data=X_valid,label=y_valid)
wlist=[(d_train,'train'),(d_valid,'eval')]
#best_params=xgb_grid.best_params_

In [17]:
# set the baseline_train the model
model = xgb.train(params=bestparm,
                  dtrain=d_train,
                  #eval_metric='mae',
                  evals=wlist,
                  num_boost_round=5000
                  #tree_method='gpu_hist'
                )

[12:32:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:576: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:31.31363	eval-rmse:31.48570
[1]	train-rmse:31.00567	eval-rmse:31.17629
[2]	train-rmse:30.70087	eval-rmse:30.86968
[3]	train-rmse:30.39880	eval-rmse:30.56598
[4]	train-rmse:30.10026	eval-rmse:30.26645
[5]	train-rmse:29.80400	eval-rmse:29.96823
[6]	train-rmse:29.51124	eval-rmse:29.67446
[7]	train-rmse:29.22069	eval-rmse:29.38287
[8]	train-rmse:28.93368	eval-rmse:29.09472
[9]	train-rmse:28.64885	eval-rmse:28.80845
[10]	train-rmse:28.36753	eval-rmse:28.52559
[11]	train-rmse:28.08854	eval-rmse:28.24506
[12]	train-rmse:27.81254	eval-rmse:27.96758
[13]	train

In [20]:
# see the test set
X_test.head()


,class,xmin,ymin,xmax,ymax,angle,depth_x,depth_y,depth_mean,depth_median,depth_max,depth_mean_trim,width,height
0,2,366.22876,183.72366,437.65915,242.04475,0.31,366,241,2.466195,2.422799,5.325973,2.425623,71.43039,58.32109
1,2,462.46075,191.03940,494.51144,221.26378,0.18,467,214,2.928864,2.930132,4.096424,2.875472,32.05069,30.22438
2,2,640.30054,176.62130,665.87164,205.29315,-1.65,664,200,5.320601,4.725276,9.226273,4.734886,25.57110,28.67185
3,5,589.00390,162.78325,621.17870,194.02880,-1.56,598,167,7.947367,7.788987,9.753765,7.820270,32.17480,31.24555
4,2,348.17825,186.73315,396.05940,213.92105,1.89,372,202,6.340028,6.058418,8.956887,6.095982,47.88115,27.18790


In [21]:
# fit to the format
d_test=xgb.DMatrix(data=X_test,label=y_test)

In [22]:
# predict result
preds= model.predict(d_test)

In [23]:
# to extract information by metric
def testing(pred,y):
    sums1=[]
    sums2=[]
    for i in range(len(preds)):
        sqr=(float(pred[i])-float(y.loc[i]))**2
        abs1=abs(float(pred[i])-float(y.loc[i]))
        sums1.append(sqr)
        sums2.append(abs1)
        
        
    rmse=(sum(sums1)/len(preds))**0.5
    mae= sum(sums2)/len(preds)
    return rmse,mae

In [ ]:
# print the test result
print('rmse =', testing(preds,y_test)[0])
print('mae =', testing(preds,y_test)[1])